In [267]:
#import modules
from flask import Flask, render_template, request, redirect, url_for, session,jsonify
from flask_bcrypt import Bcrypt
from pymongo import MongoClient
import secrets
import psycopg
from datetime import datetime
from psycopg import sql

In [268]:
#create flask connection
app = Flask(__name__)
app.secret_key = secrets.token_urlsafe(16)  # Generate a random key

#set bvrypt for later password authentication
bcrypt = Bcrypt(app)
#create mongo connection
client = MongoClient('mongodb://localhost:27017/')
db = client['user_database']
users_collection= db['users']

In [269]:
#add default password to all users in mongo(stores unmodified data- prevents password from being changed accidentaly in sql) since users now did not went through registration process
def add_passwords_to_users():
    users_without_password = users_collection.find({'password': {'$exists': False}})
    
    for user in users_without_password:
        plain_password = 'defaultPassword123'  
        hashed_password = bcrypt.hashpw(plain_password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
        
        users_collection.update_one(
            {'_id': user['_id']},
            {'$set': {'password': hashed_password}}
        )
        print(f"Updated password for {user['username']}")

add_passwords_to_users()

In [270]:
#login page creation
@app.route('/login', methods=['GET', 'POST'])
def login_page():
    if request.method == "POST":
        #seperate doctor login since only one doctor now
        if request.form['action'] == 'doctor_login':
            return redirect(url_for('api_doc'))
        
        #rewuest username and password
        username = request.form['username']
        plain_password = request.form['password']
        user = users_collection.find_one({'username': username})
        
        # Use the Bcrypt extension to check the password
        if user and bcrypt.check_password_hash(user['password'], plain_password):
            session['username'] = user['username']
            return redirect(url_for('home_page'))
        else:
            return 'Invalid username or password'
    return render_template('login.html')


In [271]:
#doctor home page
@app.route('/api_doc')
def api_doc():
    return render_template('api-doc.html')

In [272]:
#user home page if user in current database
@app.route('/')
def home_page():
    if 'username' in session:
        return render_template("api.html",username=session['username'])
    else:
        return redirect(url_for('login_page'))


In [273]:
#create postgresql connection for late ruse
def get_db_connection():
    conn = psycopg.connect(
        host="localhost",
        port='5432',
        dbname="5400api",
        user="postgres",
        password="123"
    )
    return conn


In [274]:
#info page gets data from sql and post on web
@app.route('/info')
def info_page():
    if 'username' in session:
        username = session['username']
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT * FROM user_data WHERE patient_id = %s", (username,))
        user_data = cur.fetchall()
        cur.close()
        conn.close()
        return render_template("info.html", user_data=user_data)
    else:
        # If no user is in session, redirect to the login page
        return redirect(url_for('login_page'))


In [275]:
#create user feeling table- collects real-time data for user submitte feelings
def init_db():
    conn = get_db_connection()
    cur = conn.cursor()
    # Create table if it does not exist
    cur.execute(sql.SQL("""
        CREATE TABLE IF NOT EXISTS user_feelings (
            id SERIAL PRIMARY KEY,
            username VARCHAR(255),
            status VARCHAR(50),
            date DATE,
            time TIME
        );
    """))
    conn.commit()
    cur.close()
    conn.close()

In [276]:
#record user feelings and insert into db
@app.route('/record_feeling', methods=['POST'])
def record_feeling():
    if 'username' not in session:
        return jsonify({'error': 'not authenticated'}), 401

    data = request.json
    status = data['status']
    username = session['username']
    now = datetime.now()
    date=now.date()
    time = now.strftime('%H:%M') 

    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("INSERT INTO user_feelings (username, status, date, time) VALUES (%s, %s, %s, %s)",
            (username, status, date, time))
    conn.commit()
    cur.close()
    conn.close()
    
    return jsonify({'message': 'Feeling recorded successfully'}), 200

In [277]:
#setup process
@app.before_first_request
def setup():
    init_db()

In [278]:
#status page contains all current feelings of user
@app.route('/status')
def status_page():
    if 'username' in session:
        username = session['username']
        conn = get_db_connection()
        cur = conn.cursor()
        today_date = datetime.now().date()

        # Adjusted the query to match the new table schema with separate date and time columns
        cur.execute("SELECT * FROM user_feelings WHERE username = %s AND date = %s", (username, today_date))
        user_data = cur.fetchall()
        cur.close()
        conn.close()
        return render_template("status.html", user_data=user_data)
    else:
        # If no user is in session, redirect to the login page
        return redirect(url_for('login_page'))


In [279]:
#records page contains all past feeling data for user
@app.route('/records')
def records_page():
    if 'username' in session:
        username = session['username']
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT * FROM user_feelings WHERE username = %s", (username,))
        user_data = cur.fetchall()
        cur.close()
        conn.close()
        return render_template("records.html", user_data=user_data)
    else:
        # If no user is in session, redirect to the login page
        return redirect(url_for('login_page'))

    

In [280]:
#other general features of web app
@app.route('/settings')
def settings_page():
    return render_template("settings.html")
@app.route('/support')
def support_page():
    return render_template("support.html")
#redirects user to login page once logged out
@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('login_page'))

In [281]:
#info page for doctor- contains all patient data
@app.route('/info_doc')
def info_doc():
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT * FROM user_data")
    user_data = cur.fetchall()
    cur.close()
    conn.close()
    return render_template("info-doc.html", user_data=user_data)

In [282]:
#status page contains all current predictions using different models for each patient
@app.route('/status_doc')
def status_doc():
    return render_template("status-doc.html")
#records page contains all past data-training models for each patient 
@app.route('/records_doc')
def records_doc():
    return render_template("records-doc.html")
@app.route('/settings_doc')
def settings_doc():
    return render_template("settings-doc.html")
@app.route('/support_doc')
def menu_doc():
    return render_template("menu-doc.html")
@app.route('/statusab_doc')
def statusab_doc():
    return render_template("statusab-doc.html")
@app.route('/statuslda_doc')
def statuslda_doc():
    return render_template("statuslda-doc.html")

In [283]:

#run
if __name__ == '__main__':
    app.run(host='localhost', port=1271)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:1271
Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2024 23:56:55] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [17/Apr/2024 23:56:55] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2024 23:56:55] "GET /static/css/login_style.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Apr/2024 23:56:55] "GET /highLightTitle.png HTTP/1.1" 404 -
127.0.0.1 - - [17/Apr/2024 23:56:59] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [17/Apr/2024 23:56:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2024 23:56:59] "GET /image.webp HTTP/1.1" 404 -
127.0.0.1 - - [17/Apr/2024 23:56:59] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Apr/2024 23:57:00] "GET /highLightTitle.png HTTP/1.1" 404 -
127.0.0.1 - - [17/Apr/2024 23:57:01] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2024 23:57:01] "GET /image.webp HTTP/1.1" 404 -
127.0.0.1 - - [17/Apr/2024 23:57:01] "GET /static/css/info_style.css HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2024 23:57:01] "GET /highLightTitle.png HTTP/1.1" 404 -
